<a href="https://colab.research.google.com/github/proteus21/DATA-SCIENCE-STUDY/blob/main/Machine%20Learning/10_Study_case/10_Case_studies_Customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### UCZENIE NIENADZOROWANE /  UNSUPERVISED LEARNING

#Case studies - CUSTOMER SEGMENTATION

A basic library for machine learning in Python
To install the scikit-learn library, use the command below:
```
!pip install scikit-learn
```
To update to the latest version of the scikit-learn library, use the command below:
```
!pip install --upgrade scikit-learn
```

### Contents:
1. [Import libraries](#0)
2. [Data generation ](#1)
3. [Data eksploration](#2)
4. [Determination of customer retention](#3)
5. [Retention - KMeans](#4)
6. [Retention - DBSCAN](#5)
7. [Seles](#6)
8. [Seles -KMeans](#7)
9. [Seles -DBSCAN](#8)
10. [Seles, Retention -KMeans](#8)




### <a name='0'></a> Import libraries

In [2]:
# Import the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

sns.set(font_scale=1.2)
np.random.seed(10)


### <a name='1'></a> Data generation

In [3]:
url = 'https://storage.googleapis.com/esmartdata-courses-files/ml-course/OnlineRetail.csv'
raw_data = pd.read_csv(url, encoding='latin', parse_dates=['InvoiceDate'])
data = raw_data.copy()
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


### <a name='2'></a> Data eksploration

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [6]:
data.describe(include=['object'])

,InvoiceNo,StockCode,Description,Country
count,541909,541909,540455,541909
unique,25900,4070,4223,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1114,2313,2369,495478


In [11]:
data.describe(include=['datetime'])

<ipython-input-11-8e9ac93aa46b>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe(include=['datetime'])


,InvoiceDate
count,541909
unique,23260
top,2011-10-31 14:41:00
freq,1114
first,2010-12-01 08:26:00
last,2011-12-09 12:50:00


In [12]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [13]:
data = data.dropna()
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
data['Country'].value_counts()

United Kingdom          361878
Germany                   9495
France                    8491
EIRE                      7485
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1480
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     250
Unspecified                244
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

In [16]:
tmp = data['Country'].value_counts().reset_index()
tmp.columns = ['Country', 'Count']
tmp.query("Count > 200", inplace=True)
px.bar(tmp, x='Country', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'],
       title='Frequency of purchases by country')

In [17]:
data_uk = data.query("Country == 'United Kingdom'").copy()
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [18]:
data_uk['Sales'] = data_uk['Quantity'] * data_uk['UnitPrice']
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [23]:
tmp = data_uk.groupby(data_uk['InvoiceDate'].dt.date)['CustomerID'].count().reset_index()
tmp.columns = ['InvoiceDate', 'Count']
tmp.head()

,InvoiceDate,Count
0,2010-12-01,1809
1,2010-12-02,2029
2,2010-12-03,937
3,2010-12-05,2492
4,2010-12-06,1915


In [24]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

trace1 = px.line(tmp, x='InvoiceDate', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]
trace2 = px.scatter(tmp, x='InvoiceDate', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'])['data'][0]

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)
fig.update_layout(template='plotly_dark', title='Frequency of purchases by date', width=950)
fig.show()

In [25]:
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [ ]:
tmp = data_uk.groupby(data_uk['InvoiceDate'].dt.date)['Sales'].sum().reset_index()
tmp.columns = ['InvoiceDate', 'Sales']
tmp.head()

### <a name='3'></a> Determining customer retention

In [26]:
data_user = pd.DataFrame(data['CustomerID'].unique(), columns=['CustomerID'])
data_user.head(3)

,CustomerID
0,17850.0
1,13047.0
2,12583.0


In [27]:
last_purchase = data_uk.groupby('CustomerID')['InvoiceDate'].max().reset_index()
last_purchase.columns = ['CustomerID', 'LastPurchaseDate']
last_purchase.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18 10:17:00
1,12747.0,2011-12-07 14:34:00
2,12748.0,2011-12-09 12:20:00
3,12749.0,2011-12-06 09:56:00
4,12820.0,2011-12-06 15:12:00


In [28]:
last_purchase['LastPurchaseDate'].max()

Timestamp('2011-12-09 12:49:00')

In [29]:
last_purchase['LastPurchaseDate'].min()

Timestamp('2010-12-01 09:53:00')

In [30]:
last_purchase['Retention'] = (last_purchase['LastPurchaseDate'].max() - last_purchase['LastPurchaseDate']).dt.days
last_purchase.head()

,CustomerID,LastPurchaseDate,Retention
0,12346.0,2011-01-18 10:17:00,325
1,12747.0,2011-12-07 14:34:00,1
2,12748.0,2011-12-09 12:20:00,0
3,12749.0,2011-12-06 09:56:00,3
4,12820.0,2011-12-06 15:12:00,2


In [31]:
last_purchase['Retention'].value_counts()


3      114
8       97
0       97
2       92
1       77
      ... 
250      1
350      1
369      1
209      1
219      1
Name: Retention, Length: 348, dtype: int64

In [32]:

px.histogram(last_purchase, x='Retention', template='plotly_dark',
             width=950, height=500, title='Retention', nbins=100,
             color_discrete_sequence=['#03fcb5'])

In [33]:
data_user = pd.merge(data_user, last_purchase, on='CustomerID')
data_user = data_user[['CustomerID', 'Retention']]
data_user.head()

,CustomerID,Retention
0,17850.0,301
1,13047.0,31
2,13748.0,95
3,15100.0,329
4,15291.0,25


In [34]:
px.scatter(data_user, x='CustomerID', y='Retention', template='plotly_dark', width=950,
           color_discrete_sequence=['#03fcb5'])

In [37]:
data_retention = data_user[['Retention']]
data_retention.head()

,Retention
0,301
1,31
2,95
3,329
4,25


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_user['RetentionScaled'] = scaler.fit_transform(data_retention)
data_user.head()

,CustomerID,Retention,RetentionScaled
0,17850.0,301,2.097649
1,13047.0,31,-0.596486
2,13748.0,95,0.042124
3,15100.0,329,2.377041
4,15291.0,25,-0.656356


In [39]:
px.scatter(data_user, x='CustomerID', y='RetentionScaled', template='plotly_dark', width=950,
           color_discrete_sequence=['#03fcb5'])

In [40]:

data_retention_scaled = data_user[['RetentionScaled']]
data_retention_scaled.head()

,RetentionScaled
0,2.097649
1,-0.596486
2,0.042124
3,2.377041
4,-0.656356


### <a name='4'></a> Retention - KMeans

In [44]:
from sklearn.cluster import KMeans

wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000, n_init=1)
    kmeans.fit(data_retention_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss

,NumberOfClusters,WCSS
0,1.0,3950.000000
1,2.0,814.442559
2,3.0,388.020760
3,4.0,221.451330
4,5.0,136.657761
5,6.0,99.155630
6,7.0,75.436135
7,8.0,54.770694
8,9.0,45.716061


In [45]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', title='WCSS',
        width=950, color_discrete_sequence=['#03fcb5'])

In [47]:
kmeans = KMeans(n_clusters=3, max_iter=1000, n_init=1)
kmeans.fit(data_retention_scaled)

KMeans(max_iter=1000, n_clusters=3, n_init=1)

In [48]:

data_user['Cluster'] = kmeans.labels_
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster
0,17850.0,301,2.097649,1
1,13047.0,31,-0.596486,0
2,13748.0,95,0.042124,2
3,15100.0,329,2.377041,1
4,15291.0,25,-0.656356,0


In [53]:
tmp = data_user.groupby('Cluster')['Retention'].describe()
tmp

,count,mean,std,min,25%,50%,75%,max
Cluster,,,,,,,,
0,2651.0,30.307808,24.994104,0.0,9.0,24.0,49.00,91.0
1,564.0,293.684397,45.612310,224.0,255.0,286.5,326.75,373.0
2,735.0,153.185034,38.155512,92.0,119.0,153.0,185.00,222.0


In [54]:
tmp = tmp['mean'].reset_index()
tmp.columns = ['Cluster', 'MeanRetention']
px.bar(tmp, x='Cluster', y='MeanRetention', template='plotly_dark', width=950,
       height=400, color_discrete_sequence=['#03fcb5'])

In [56]:

px.scatter(data_user, x='CustomerID', y='Retention', color='Cluster', template='plotly_dark',
           width=950, title='Cluster visualisation')

In [57]:

tmp = data_user['Cluster'].value_counts().reset_index()
tmp.columns = ['Cluster', 'Count']
px.bar(tmp, x='Cluster', y='Count', template='plotly_dark', color_discrete_sequence=['#03fcb5'], width=950,
       title='Frequency distribution of clusters')

### <a name='5'></a>Retention- DBSCAN

In [58]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.03, min_samples=5)
dbscan.fit(data_retention_scaled)
clusters = dbscan.labels_
data_user['Cluster'] = clusters
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster
0,17850.0,301,2.097649,0
1,13047.0,31,-0.596486,1
2,13748.0,95,0.042124,1
3,15100.0,329,2.377041,0
4,15291.0,25,-0.656356,1


In [59]:
px.scatter(data_user, x='CustomerID', y='Retention', color='Cluster', template='plotly_dark', width=950,
           title='Cluster visualisation')

### <a name='4'></a>Sales

In [60]:
data_sales = data_uk.groupby('CustomerID')['Sales'].sum().reset_index()
data_sales.head()

,CustomerID,Sales
0,12346.0,0.00
1,12747.0,4196.01
2,12748.0,29072.10
3,12749.0,3868.20
4,12820.0,942.34


In [61]:

data_user = pd.merge(data_user, data_sales, on='CustomerID')
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales
0,17850.0,301,2.097649,0,5288.63
1,13047.0,31,-0.596486,1,3079.10
2,13748.0,95,0.042124,1,948.25
3,15100.0,329,2.377041,0,635.10
4,15291.0,25,-0.656356,1,4596.51


In [62]:
scaler = StandardScaler()
data_user['SalesScaled'] = scaler.fit_transform(data_user[['Sales']])
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,1,3079.10,0.208577
2,13748.0,95,0.042124,1,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,1,4596.51,0.440321


In [63]:
px.scatter(data_user, x='CustomerID', y='Sales', template='plotly_dark',
           color_discrete_sequence=['#03fcb5'], title='Sales  by customer')


In [64]:
px.scatter(data_user, x='CustomerID', y='SalesScaled', template='plotly_dark',
           color_discrete_sequence=['#03fcb5'], title='Sales by customer - scaled data')


In [65]:
data_sales_scaled = data_user[['SalesScaled']]
data_sales_scaled.head()

,SalesScaled
0,0.546024
1,0.208577
2,-0.116854
3,-0.164680
4,0.440321


### <a name='6'></a>Sales - KMeans

In [67]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000, n_init=1)
    kmeans.fit(data_sales_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss

,NumberOfClusters,WCSS
0,1.0,3950.000000
1,2.0,1685.972621
2,3.0,595.548482
3,4.0,354.495212
4,5.0,220.964742
5,6.0,165.509178
6,7.0,105.250718
7,8.0,76.836819
8,9.0,53.549683


In [68]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', color_discrete_sequence=['#03fcb5'],
        width=950, title='WCSS')

In [69]:

kmeans = KMeans(n_clusters=3, max_iter=1000)
kmeans.fit(data_sales_scaled)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(max_iter=1000, n_clusters=3)

In [70]:

data_user['Cluster'] = kmeans.labels_
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [71]:
kmeans.cluster_centers_

array([[-0.06065062],
       [ 6.31619638],
       [33.63689221]])

In [72]:

px.scatter(data_user, x='CustomerID', y='SalesScaled', color='Cluster', template='plotly_dark', width=950,
           title='Cluster visualisation - Data Scaled')

## <a name='7'></a>Sales - DBSCAN

In [73]:
dbscan = DBSCAN(eps=0.5, min_samples=7)
dbscan.fit(data_sales_scaled)
clusters = dbscan.labels_
data_user['Cluster'] = clusters
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [74]:
px.scatter(data_user, x='CustomerID', y='Sales', color='Cluster', template='plotly_dark', width=950,
           title='DBSCAN - Cluster visualisation')

## <a name='8'></a>Retention, Sales - KMeans

In [75]:
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,0,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,0,948.25,-0.116854
3,15100.0,329,2.377041,0,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [76]:
px.scatter(data_user, x='RetentionScaled', y='SalesScaled', template='plotly_dark', width=950,
           title='Retention vs. Sales')

In [77]:
data_scaled = data_user[['RetentionScaled', 'SalesScaled']]
data_scaled.head()

,RetentionScaled,SalesScaled
0,2.097649,0.546024
1,-0.596486,0.208577
2,0.042124,-0.116854
3,2.377041,-0.164680
4,-0.656356,0.440321


In [79]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, max_iter=1000, n_init=1)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

wcss = pd.DataFrame(data=np.c_[range(1, 10), wcss], columns=['NumberOfClusters', 'WCSS'])
wcss

,NumberOfClusters,WCSS
0,1.0,7900.000000
1,2.0,4714.243847
2,3.0,2629.432962
3,4.0,1383.090602
4,5.0,953.202349
5,6.0,730.321831
6,7.0,598.936212
7,8.0,447.061034
8,9.0,360.770396


In [80]:
px.line(wcss, x='NumberOfClusters', y='WCSS', template='plotly_dark', color_discrete_sequence=['#03fcb5'], width=950,
        title='WCSS')

In [82]:
kmeans = KMeans(n_clusters=5, max_iter=1000, n_init=1)
kmeans.fit(data_scaled)

KMeans(max_iter=1000, n_clusters=5, n_init=1)

In [83]:
data_user['Cluster'] = kmeans.labels_
data_user['Cluster'] = data_user['Cluster'].astype(str)
data_user.head()

,CustomerID,Retention,RetentionScaled,Cluster,Sales,SalesScaled
0,17850.0,301,2.097649,1,5288.63,0.546024
1,13047.0,31,-0.596486,0,3079.10,0.208577
2,13748.0,95,0.042124,4,948.25,-0.116854
3,15100.0,329,2.377041,1,635.10,-0.164680
4,15291.0,25,-0.656356,0,4596.51,0.440321


In [84]:
px.scatter(data_user, x='RetentionScaled', y='SalesScaled', color='Cluster', template='plotly_dark', width=950,
           title='KMeans - cluster visualisation')

In [85]:
centroids = kmeans.cluster_centers_
centroids

array([[-6.01736815e-01, -3.97778210e-04],
       [ 2.02465229e+00, -2.02955850e-01],
       [-8.48160918e-01,  6.31619638e+00],
       [-8.70889226e-01,  3.36368922e+01],
       [ 6.18220917e-01, -1.65274327e-01]])

In [86]:
fig = px.scatter(data_user, x='RetentionScaled', y='SalesScaled', color='Cluster', template='plotly_dark', width=900,
                 title='KMeans - Cluster visualisation + centroids')
fig.add_trace(go.Scatter(x=centroids[:, 0], y=centroids[:, 1], mode='markers', marker_symbol='star',
                         marker_size=10, marker_color='white', showlegend=False))

In [87]:
desc = data_user.groupby('Cluster')[['Retention', 'Sales']].describe()
desc

Retention                                                              \
            count        mean        std    min     25%    50%     75%    max   
Cluster                                                                         
0          2619.0   30.519664  24.930015    0.0    9.00   24.0   49.00   99.0   
1           564.0  293.684397  45.612310  224.0  255.00  286.5  326.75  373.0   
2            27.0    5.777778   9.082245    0.0    0.00    2.0    6.50   38.0   
3             2.0    3.500000   4.949747    0.0    1.75    3.5    5.25    7.0   
4           738.0  152.903794  38.327273   90.0  119.00  153.0  185.00  222.0   

          Sales                                                       \
          count           mean           std        min          25%   
Cluster                                                                
0        2619.0    1710.071987   2333.685303   -1165.30     396.4050   
1         564.0     384.475567   1044.072045   -1192.20     137.9100   
2          27.0   43070.445185  15939.249588   25748.35   28865.4900   
3           2.0  221960.330000  48759.481478  187482.17  204721.2500   
4         738.0     630.796695    769.123906   -4287.63     203.8625   

                                            
               50%          75%        max  
Cluster                                     
0           911.26    2051.8900   21086.30  
1           249.93     391.9725   21535.90  
2         36351.42   53489.7900   88125.38  
3        221960.33  239199.4100  256438.49  
4           392.52     777.4825    7092.06

In [88]:
tmp = pd.merge(desc['Retention'][['count', 'mean']].reset_index(), desc['Sales'][['mean']].reset_index(), on='Cluster',
         suffixes=('_Retention', '_Sales'))
tmp

,Cluster,count,mean_Retention,mean_Sales
0,0,2619.0,30.519664,1710.071987
1,1,564.0,293.684397,384.475567
2,2,27.0,5.777778,43070.445185
3,3,2.0,3.500000,221960.330000
4,4,738.0,152.903794,630.796695


In [89]:
px.bar(tmp, x='count', y='Cluster', hover_data=['mean_Retention', 'mean_Sales'], template='plotly_dark',
       width=950, orientation='h', title='Cluster distribuation')